In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

(1)
Read the data.  
Change your index to the date column.  
Sort the df by indices. 
Drop the old date column.  
Rename any column you want to make things easier later.  
Display the df.

In [ ]:
import pandas as pd
import numpy as np
import datetime 

df = pd.read_csv('/kaggle/input/apple-aapl-historical-stock-data/HistoricalQuotes.csv')
df['Date'] = df['Date'].str.replace('/', '')
df['Date'] = pd.to_datetime(df['Date'], format='%m%d%Y')
df = df.sort_values('Date')
df = df.rename({'Date': 'date', ' Close/Last': 'close', ' Volume': 'volume', ' Open': 'open', ' High': 'high', ' Low': 'low'}, axis='columns')
df = df.reset_index().drop(columns = 'index')
print(df)

(2) Get rid of the $ sign and make each column a number.  
We many not use each column for this particular task, but it would be useful in a future ML exercise for prediction.  
Display the df.

In [ ]:
df['open'] = df.open.str.replace('$', '')
df['close'] = df.close.str.replace('$', '')
df['high'] = df.high.str.replace('$', '')
df['low'] = df.low.str.replace('$', '')
df['open'] = df['open'].astype('float')
df['close'] = df['close'].astype('float')
df['high'] = df['high'].astype('float')
df['low'] = df['low'].astype('float')
print(df)

(3) Add a new column representing the daily increase or decrease in the closing value of the stock.  
For instance, on the second date, the stock went down by -0.0200.  
On the third date, the stock went up by 0.0686.  
Display the df.

In [ ]:
df['change'] = df['close'] - df['open']
print(df)

(4) Add a new column with the rolling mean, use a 20 day window.  
Also add a column for the rolling standard deviation with the same 20 day window.  
Display the df.

In [ ]:
df['rolling_mean'] = df.close.rolling(20).mean()
df['rolling_std'] = df.close.rolling(20).std()
print(df)

(5) Add two new columns, one for the upper and one for the lower Bollinger bands.  
The typical Bollinger bands are calculated by adding two standard deviations away from the mean for the upper,  
and subtracting two standard deviations away from the mean for the lower.  
Remove all the NaN rows now.  
Display the df.

In [ ]:
df['upper_band'] = df['rolling_mean'] + 2 * df['rolling_std']
df['lower_band'] = df['rolling_mean'] - 2 * df['rolling_std']
df = df.dropna().reset_index().drop(columns = 'index')
print(df)

(6) Create a smaller dataframe using only data over a 6 month period from Jan 1, 2014 to July 1, 2014.  
Use matplotlib to plot the following: Closing Value, Rolling Mean, Upper Bollinger, Lower Bollinger.   
Use a different color for each plot.  
Use the function fill_between to color between the Upper/Lower bands with a green color using alpha of 0.1 so it is see through.  
Show your plot.

In [ ]:
import matplotlib.pyplot as plt

plt.figure( figsize=(20, 20) )
smdf = df.loc[ (df['date'] >= '2014-01-01') & (df['date'] <= '2014-07-01') ]
smdf = smdf.reset_index().drop(columns = 'index')
graph = smdf.plot(x = 'date', y = ['close', 'rolling_mean', 'upper_band', 'lower_band'])
graph.fill_between(x = smdf['date'], y1 = smdf['lower_band'], y2 = smdf['upper_band'], alpha = 0.1, color = 'green')

(7) One theory of stocks is to buy when the current value is below the lower Bollinger band  
and sell when the current value is above the upper Bollinger band.  
This does get a bit trickier because it is potentially better to buy or sell after the moment  
the stock rebounds inside the Bollinger bands, that way you do not buy or sell a stock that is in  
a gigantic freefall or a huge spike, but we will ignore that.  
Create a new column called Buy that is true each time the current closing value is lower than the lower band  
and another column for Sell that is true each time the current closing value is higher than the upper band.  
Display your df.

In [ ]:
smdf['buy'] = smdf['close'] <= smdf['lower_band']
smdf['sell'] = smdf['close'] >= smdf['upper_band']
print(smdf)

(8) Display all of the rows where Buy is true.

In [ ]:
print( smdf.loc[ smdf['buy'] == True ] )

(9) Display all of the rows where Sell is true.

In [ ]:
print( smdf.loc[ smdf['sell'] == True ] )

(10) Display the final closing value of the last day.

In [ ]:
print(smdf['close'].iloc[[-1]])

(11) You can calculate this by hand or you can think through the code to automate this task.  
Automating the task is ideal, but for this small enough set it will take a lot more time  
to figure out the logic.  
  
How much money would you have gained or lost in the following two situations:

a) You bought 500 shares of Apple on January 1st of 2014 and sold it on July 1st of 2014.

b) You bought 500 shares of Apple the first day your 'buy' column told you to. 
You sold the 500 shares the next date after that when your 'sell' column told you to. 
You keep repeating this process, either buying 500 shares or selling the 500 shares you own. 
In this exercise, you may never have more than 500 shares or less than 0.  
If you have 500 shares on July 1st, you will sell it for its closing cost.  
(Not realistic, just an exercise).

In [ ]:
#a. 
#Get the initial cost of stock; 500 * price of stock on 2014-01-01 ($79.02) = $39,510
print(df['close'].loc[ df['date'] == '2014-01-02' ])
#subtract the initial price of stock from the value of the stock at July 1st
print(df['close'].loc[ df['date'] == '2014-07-01' ])
#(500 * 93.52) - 39,510 = $7,250 Gained 

#b. 
 # 500 * 77.2828 = 38640
 # (500 * 77.85) - 38640 = $285 Gain on 03-25
 
 #$285 was the profit because there were no more buy periods after January of 2014